In [10]:
import json

from sklearn.model_selection import train_test_split 

In [9]:
!pip3 install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 26.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 27.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 25.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


## A script to format regulat export to one that can be used for ultralistic yolo training

In [4]:
label_file = 'dartnet_data/labels.json'

labels = {}
with open(label_file, 'r') as f:
    labels = json.load(f)
labels = labels['labels']
labels

[{'image_filename': 'dartnet_data/dart_0_1.png',
  'series_id': 0,
  'annotations': [{'bbox': [814, 505], 'label': 'dart'},
   {'bbox': [232, 425], 'label': 'anchor'},
   {'bbox': [495, 195], 'label': 'anchor'},
   {'bbox': [750, 442], 'label': 'anchor'},
   {'bbox': [487, 643], 'label': 'anchor'}]},
 {'image_filename': 'dartnet_data/dart_0_2.png',
  'series_id': 0,
  'annotations': [{'bbox': [814, 505], 'label': 'dart'},
   {'bbox': [213, 432], 'label': 'dart'},
   {'bbox': [232, 425], 'label': 'anchor'},
   {'bbox': [495, 195], 'label': 'anchor'},
   {'bbox': [750, 442], 'label': 'anchor'},
   {'bbox': [487, 643], 'label': 'anchor'}]},
 {'image_filename': 'dartnet_data/dart_0_3.png',
  'series_id': 0,
  'annotations': [{'bbox': [814, 505], 'label': 'dart'},
   {'bbox': [213, 432], 'label': 'dart'},
   {'bbox': [779, 553], 'label': 'dart'},
   {'bbox': [232, 425], 'label': 'anchor'},
   {'bbox': [495, 195], 'label': 'anchor'},
   {'bbox': [750, 442], 'label': 'anchor'},
   {'bbox': [4

In [8]:
ids = set(map(lambda x: x['series_id'], labels))
ids = list(ids)

In [11]:
ratio = 0.2
train_ids, val_ids = train_test_split(ids, test_size=ratio)

In [18]:
from PIL import Image

fn = labels[0]['image_filename']
i = Image.open(fn)
width, height = i.size
width, height

(990, 730)

In [27]:
import os
import shutil

from tqdm.notebook import tqdm

classs_mappig = {
    'anchor': 0,
    'dart': 1
}

def build_label(box_size, pos_x, pos_y, img_width, img_height):
    span = box_size / 2 
    
    left = max(0.1, pos_x - span)
    top = max(0.1, pos_y - span)

    # norm
    left, top = left / img_width, top / img_height
    width = box_size / img_width
    height = box_size / img_height
    return left, top, width, height

def get_fn(path):
    return path.split('/')[-1]

def copy_img(label, out_dir):
    img_path = label['image_filename']
    
    out_path = out_dir + '/' + get_fn(img_path)

    # Create missing directories if they don't exist
    destination_directory = os.path.dirname(out_path)
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)
        
    shutil.copy(img_path, out_path)

def build_label_file(label, out_dir, img_size, box_size=50):
    img_fn = get_fn(label['image_filename'])
    label_fn = '.'.join(img_fn.split('.')[0:-1] ) + '.txt'
    out_path = out_dir + '/' + label_fn

    img_width, img_height = img_size
    annotations = []
    for a in label['annotations']:
        c = classs_mappig[a['label']]
        pos_x, pos_y = a['bbox']
        left, top, width, height = build_label(box_size, pos_x, pos_y, img_width, img_height)
        label_str = f'{c} {left} {top} {width} {height}'
        annotations.append(label_str)
    with open(out_path, 'w') as f:
        content = '\n'.join(annotations)
        f.write(content)
        
out_dir_base = 'data_yolo'
out_dir_train = out_dir_base + '/train'
out_dir_val = out_dir_base + '/val'


for label in tqdm(labels):
    series_id = label['series_id']
    is_train = not series_id in val_ids
    out_dir = out_dir_train if is_train else out_dir_val

    copy_img(label, out_dir)
    build_label_file(label, out_dir, (width, height), box_size=70)

  0%|          | 0/300 [00:00<?, ?it/s]

In [20]:
!pip3 install tqdm

  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
